## Easy

Среднее считается по определению как сумма элементов/число элементов
Медиана считается по определению. Если число элементов чётно, выбирается центральный. Иначе среднее арифметическое двух центральных.
Чтобы посчитать моду, сначала ищем максимальное число вхождений элемента в список, потом выбираем произвольный элемент с таким вхождением.

In [1]:
def func(lst):
    res = {}
    if not len(lst):
        return res
    
    res["mean"] = sum(lst) / len(lst)
    
    mid = len(lst) // 2
    if len(lst) % 2:
        res["median"] = sorted(lst)[mid]
    else:
        sorted_lst = sorted(last)
        res["median"] = (sorted_lst[mid - 1] + sorted_lst[mid]) / 2

    max_cnt = max([lst.count(i) for i in lst])
    res["mode"] = list(filter(lambda x: lst.count(x) == max_cnt, lst))[0]

    return res

In [2]:
func([0, 1, 1, 10, 5, 4, 3])

{'mean': 3.4285714285714284, 'median': 3, 'mode': 1}

## Medium

Разбиваем строку по формату csv, записываем данные. Числа приводим к типу int, список в последнем атрибуте парсим дополнительно.

In [3]:
def parse_csv(content):
    data = [i.split(',') for i in content.split('\n')]
    res = [{} for _ in range(len(data) - 2)]
    for i, character in enumerate(data[1:-1]):
        res[i][""] = int(character[0])
        res[i]["attack_type"] = character[1]
        res[i]["id"] = int(character[2])
        res[i]["legs"] = int(character[3])
        res[i]["localized_name"] = character[4]
        res[i]["name"] = character[5]
        res[i]["primary_attr"] = character[6]
        res[i]["roles"] = [s.strip("' ") for s in ','.join(character[7:]).strip('"[]').split(',')]
    return res

Читаем данные и парсим

In [4]:
with open("dota_hero_stats.csv") as file:
    content = file.read()

characters = parse_csv(content)
characters

[{'': 0,
  'attack_type': 'Melee',
  'id': 1,
  'legs': 2,
  'localized_name': 'Anti-Mage',
  'name': 'npc_dota_hero_antimage',
  'primary_attr': 'agi',
  'roles': ['Carry', 'Escape', 'Nuker']},
 {'': 1,
  'attack_type': 'Melee',
  'id': 2,
  'legs': 2,
  'localized_name': 'Axe',
  'name': 'npc_dota_hero_axe',
  'primary_attr': 'str',
  'roles': ['Initiator', 'Durable', 'Disabler', 'Jungler']},
 {'': 2,
  'attack_type': 'Ranged',
  'id': 3,
  'legs': 4,
  'localized_name': 'Bane',
  'name': 'npc_dota_hero_bane',
  'primary_attr': 'int',
  'roles': ['Support', 'Disabler', 'Nuker', 'Durable']},
 {'': 3,
  'attack_type': 'Melee',
  'id': 4,
  'legs': 2,
  'localized_name': 'Bloodseeker',
  'name': 'npc_dota_hero_bloodseeker',
  'primary_attr': 'agi',
  'roles': ['Carry', 'Disabler', 'Jungler', 'Nuker', 'Initiator']},
 {'': 4,
  'attack_type': 'Ranged',
  'id': 5,
  'legs': 2,
  'localized_name': 'Crystal Maiden',
  'name': 'npc_dota_hero_crystal_maiden',
  'primary_attr': 'int',
  'roles'

Ищем максимальное число ног и находим персонажа с этим значением.

In [5]:
max_legs = max(i["legs"] for i in characters)
list(filter(lambda x: x["legs"] == max_legs, characters))[0]

{'': 59,
 'attack_type': 'Melee',
 'id': 61,
 'legs': 8,
 'localized_name': 'Broodmother',
 'name': 'npc_dota_hero_broodmother',
 'primary_attr': 'agi',
 'roles': ['Carry', 'Pusher', 'Escape', 'Nuker']}

## Hard

Посчитаем сходство по отдельным параметрам:
- attack_type: может совпадать или не совпадать - 0 или 1
- legs: разница в числе ног не больше 8 - доля разности от 8 
- primary_attr: может совпадать или не совпадать - 0 или 1
- roles: двум персонажам соответствуют два множества ролей - отношение размера их пересечения к размеру их объединения

Сложим полученные значения с какими-то весами. Приоритет атрибутов неизвестен, так что все веса будут равны 0.25

В итоге метрика принимает значения с интервала (0; 1), у похожих персонажей метрика ближе к 1.

In [6]:
def cmp(x, y):
    cmp_attack_type = x["attack_type"] == y["attack_type"]
    cmp_legs = 1 - abs(x["legs"] - y["legs"]) / 8
    cmp_primary_attr = x["primary_attr"] == y["primary_attr"]
    roles1 = set(x["roles"])
    roles2 = set(y["roles"])
    cmp_roles = len(roles1 & roles2) / len(roles1 | roles2)
    return 0.33 * (cmp_attack_type + cmp_primary_attr + cmp_roles) + 0.01 * cmp_legs

Перебираем все двойки различных персонажей, запоминая лучшую.

In [7]:
res = (None, None)
best_cmp = -1
for i, x in enumerate(characters):
    for j, y in enumerate(characters[i + 1:]):
        cur_cmp = cmp(x, y)
        if cur_cmp > best_cmp:
            best_cmp = cur_cmp
            res = (x, y)
res, best_cmp

(({'': 4,
   'attack_type': 'Ranged',
   'id': 5,
   'legs': 2,
   'localized_name': 'Crystal Maiden',
   'name': 'npc_dota_hero_crystal_maiden',
   'primary_attr': 'int',
   'roles': ['Support', 'Disabler', 'Nuker', 'Jungler']},
  {'': 88,
   'attack_type': 'Ranged',
   'id': 90,
   'legs': 2,
   'localized_name': 'Keeper of the Light',
   'name': 'npc_dota_hero_keeper_of_the_light',
   'primary_attr': 'int',
   'roles': ['Support', 'Nuker', 'Disabler', 'Jungler']}),
 1.0)